<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/PJH/GraphCodeBERT_dacon2__random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import os
import random
from itertools import combinations, product
import re
import sklearn
from sklearn.model_selection import train_test_split
import torch
torch.set_float32_matmul_precision('high')
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Seed 고정 함수
def seed_everything(seed: int = 42, contain_cuda: bool = False):
  os.environ['PYTHONHASHSEED'] = str(seed)
  random.seed(seed)
  np.random.seed(seed)

  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  print(f"Seed set as {seed}")

seed = 42
seed_everything(seed)

Seed set as 42


In [7]:
# CUDA 사용 가능 여부 확인 및 GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [8]:
def preprocess_and_remove_extras(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read()

    content = re.sub(re.compile("/\*.*?\*/", re.DOTALL), "", content) # 멀티 라인 주석 제거
    content = re.sub(re.compile("//.*?\n"), "", content) # 싱글 라인 주석 제거
    content = re.sub(re.compile("#include <.*?>\n"), "", content) # angle brackets를 사용하는 include 제거
    content = re.sub(re.compile("#include \".*?\"\n"), "", content) # double quotes를 사용하는 include 제거
    content = re.sub(re.compile("#define .*?\n"), "", content) # 매크로 정의 제거
    content = re.sub(re.compile("[\t ]+"), " ", content) # 공백 및 탭 정리
    content = re.sub(re.compile("\n\s*\n"), "\n", content)# 여러 줄바꿈을 하나로

    # 공백이 아닌 줄만 선택하여 리스트로 만든 후, 문자열로 결합
    processed_script = '\n'.join([line.strip() for line in content.splitlines() if line.strip()])

    return processed_script

In [9]:
#  전처리 적용
#  이미 create_train_data 만들 때 적용했지만, test.csv와, sample_train.csv에도 같은 전처리를 하기 위해 다시 정의
def remove_extras(code):
    code = re.sub(re.compile("/\*.*?\*/", re.DOTALL), "", code) # 멀티 라인 주석 제거
    code = re.sub(re.compile("//.*?\n"), "", code) # 싱글 라인 주석 제거
    code = re.sub(re.compile("#include <.*?>\n"), "", code)  # angle brackets를 사용하는 include 제거
    code = re.sub(re.compile("#include \".*?\"\n"), "", code)  # double quotes를 사용하는 include 제거
    code = re.sub(re.compile("#define .*?\n"), "", code)  # 매크로 정의 제거
    code = re.sub(re.compile("[\t ]+"), " ", code)  # 탭과 여러 공백을 하나의 공백으로
    code = re.sub(re.compile("\n\s*\n"), "\n", code)  # 여러 줄바꿈을 하나로

    return code.strip()

In [10]:
class CodeDataset(Dataset):
    def __init__(self, tokenizer, data, max_length=512, include_labels=True):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        code = remove_extras(self.data.iloc[idx]['code'])  # 특정 인덱스의 코드를 가져와 추가 요소를 제거합니다.

        inputs = self.tokenizer(
            code,
            padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        inputs = {key: val.squeeze() for key, val in inputs.items()}

        label = torch.tensor(self.data.iloc[idx]['labels'], dtype=torch.long)  # 해당 코드의 정답 라벨을 가져옵니다.
        inputs['labels'] = label

        return inputs

In [12]:
filepath_train = '/content/drive/MyDrive/final_train5.csv'
filepath_val = '/content/drive/MyDrive/final_val2.csv'
train_df = pd.read_csv(filepath_train)
val_df = pd.read_csv(filepath_val)

In [13]:
train_df = train_df.drop(columns=['Unnamed: 0'])
val_df = val_df.drop(columns=['Unnamed: 0'])
train_df = train_df.drop(columns=['vul'])
val_df = val_df.drop(columns=['vul'])

In [14]:
# GraphCodeBERT 모델 및 토크나이저 로드
model_name = "microsoft/graphcodebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.truncation_side = "left"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=11).to(device)

# 훈련 세트와 검증 세트에 대한 데이터셋 생성
train_dataset = CodeDataset(tokenizer, train_df, max_length=512)
val_dataset = CodeDataset(tokenizer, val_df, max_length=512, include_labels=True)

# 데이터 로더 준비
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# 파인 튜닝을 위한 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=2e-5)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import f1_score, classification_report
from tqdm import tqdm
import pandas as pd

# 설정
k = 5  # K-Fold 교차 검증에서의 fold 수
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# 모델 및 토크나이저 설정
model_name = "microsoft/graphcodebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 전체 데이터셋을 준비
total_df = pd.read_csv("/content/drive/MyDrive/Labels_Safe_MSR+Juliet_random.csv")
# 결과를 저장할 리스트
all_train_losses = []
all_val_losses = []
all_train_f1_scores = []
all_val_f1_scores = []

# 각 fold에 대한 훈련 시작
fold = 0
for train_index, val_index in kf.split(total_df):
    fold += 1
    train_data = total_df.iloc[train_index]
    val_data = total_df.iloc[val_index]

    train_dataset = CodeDataset(tokenizer, train_data, max_length=512)
    val_dataset = CodeDataset(tokenizer, val_data, max_length=512)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

    # 모델 재설정 (각 fold에서 새로운 모델을 사용)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=11)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=2e-5)

    train_losses = []
    val_losses = []
    train_f1_scores = []
    val_f1_scores = []

    # 얼리스탑 설정
    patience = 5
    best_val_loss = float('inf')
    early_stopping_counter = 0

    for epoch in range(100):  # 에폭 수 변경 가능
        model.train()
        total_train_loss = 0
        all_predictions_train = []
        all_labels_train = []
        for batch in tqdm(train_loader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_train_loss += loss.item()

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            labels = batch["labels"]

            all_predictions_train.extend(predictions.cpu().numpy())
            all_labels_train.extend(labels.cpu().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_loss = total_train_loss / len(train_loader)
        train_f1 = f1_score(all_labels_train, all_predictions_train, average='weighted')

        model.eval()
        total_val_loss = 0
        all_predictions_val = []
        all_labels_val = []
        for batch in tqdm(val_loader):
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)
            loss = outputs.loss
            total_val_loss += loss.item()

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            labels = batch["labels"]

            all_predictions_val.extend(predictions.cpu().numpy())
            all_labels_val.extend(labels.cpu().numpy())

        val_loss = total_val_loss / len(val_loader)
        val_f1 = f1_score(all_labels_val, all_predictions_val, average='weighted')

        # 클래스별 F1-score 계산
        class_report = classification_report(all_labels_val, all_predictions_val, digits=4, output_dict=False)
        print(f"Fold {fold}, Epoch {epoch+1}, Training Loss: {train_loss}, Train F1 Score: {train_f1}, Validation Loss: {val_loss}, Validation F1 Score: {val_f1}")
        print("Classification Report:")
        print(class_report)

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_f1_scores.append(train_f1)
        val_f1_scores.append(val_f1)

        # 얼리스탑 로직
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1} for fold {fold}")
                break

    # 폴드 결과를 저장
    all_train_losses.append(train_losses)
    all_val_losses.append(val_losses)
    all_train_f1_scores.append(train_f1_scores)
    all_val_f1_scores.append(val_f1_scores)

# 그래프로 나타내기
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
for i in range(k):
    plt.plot(range(1, len(all_train_losses[i]) + 1), all_train_losses[i], label=f'Train Loss Fold {i+1}')
    plt.plot(range(1, len(all_val_losses[i]) + 1), all_val_losses[i], label=f'Val Loss Fold {i+1}')
plt.title('Training and Validation Loss per Fold')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
for i in range(k):
    plt.plot(range(1, len(all_train_f1_scores[i]) + 1), all_train_f1_scores[i], label=f'Train F1 Score Fold {i+1}')
    plt.plot(range(1, len(all_val_f1_scores[i]) + 1), all_val_f1_scores[i], label=f'Val F1 Score Fold {i+1}')
plt.title('F1 Score per Fold')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.legend()

plt.tight_layout()
plt.show()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 206/206 [00:38<00:00,  5.31it/s]


Fold 1, Epoch 1, Training Loss: 0.8494475497250876, Train F1 Score: 0.7062850391256588, Validation Loss: 0.5825486937040482, Validation F1 Score: 0.7883649124308776
Classification Report:
              precision    recall  f1-score   support

           0     0.9048    0.8702    0.8872       601
           1     0.5501    0.8100    0.6552       400
           2     0.8549    0.9947    0.9195       379
           3     0.9448    0.8228    0.8796       333
           4     0.8456    0.9543    0.8966       350
           5     0.9823    0.9327    0.9568       297
           6     0.3955    0.5212    0.4497       236
           7     0.9741    0.8210    0.8910       229
           8     0.9796    0.5549    0.7085       173
           9     0.7674    0.2102    0.3300       157
          10     0.8197    0.3968    0.5348       126

    accuracy                         0.7921      3281
   macro avg     0.8199    0.7172    0.7372      3281
weighted avg     0.8229    0.7921    0.7884      3281


100%|██████████| 206/206 [00:38<00:00,  5.31it/s]


Fold 1, Epoch 2, Training Loss: 0.4711221496301998, Train F1 Score: 0.8297271307442137, Validation Loss: 0.4956043277713425, Validation F1 Score: 0.8256538906830188
Classification Report:
              precision    recall  f1-score   support

           0     0.8406    0.8952    0.8670       601
           1     0.6820    0.7775    0.7266       400
           2     0.8549    0.9947    0.9195       379
           3     0.8680    0.8889    0.8783       333
           4     0.8902    0.8571    0.8734       350
           5     0.9720    0.9360    0.9537       297
           6     0.7191    0.5424    0.6184       236
           7     0.9785    0.7948    0.8771       229
           8     0.8592    0.7052    0.7746       173
           9     0.6727    0.7070    0.6894       157
          10     0.6697    0.5794    0.6213       126

    accuracy                         0.8278      3281
   macro avg     0.8188    0.7889    0.7999      3281
weighted avg     0.8302    0.8278    0.8257      3281


100%|██████████| 206/206 [00:38<00:00,  5.32it/s]


...

Fold 3, Epoch 4, Training Loss: 0.2548132999828505, Train F1 Score: 0.9050089384947961, Validation Loss: 0.4360869914861169, Validation F1 Score: 0.8635188464692448
Classification Report:
              precision    recall  f1-score   support

           0     0.9465    0.8844    0.9144       640
           1     0.7275    0.8787    0.7960       404
           2     0.8865    1.0000    0.9398       328
           3     0.8902    0.9086    0.8993       339
           4     0.8714    0.8944    0.8828       341
           5     0.9669    0.9331    0.9497       344
           6     0.7259    0.6413    0.6810       223
           7     0.9484    0.9018    0.9245       224
           8     0.8408    0.8098    0.8250       163
           9     0.7152    0.7105    0.7129       152
          10     0.8718    0.5574    0.6800       122

    accuracy                         0.8646      3280
   macro avg     0.8537    0.8291    0.8368      3280
weighted avg     0.8684    0.8646    0.8635      3280


100%|██████████| 205/205 [00:38<00:00,  5.34it/s]


Fold 3, Epoch 5, Training Loss: 0.19556705185581436, Train F1 Score: 0.9239765516048397, Validation Loss: 0.451563662215613, Validation F1 Score: 0.8657184637927846
Classification Report:
              precision    recall  f1-score   support

           0     0.9384    0.8812    0.9089       640
           1     0.7168    0.8960    0.7965       404
           2     0.8839    0.9512    0.9163       328
           3     0.9792    0.8348    0.9013       339
           4     0.8346    0.9619    0.8937       341
           5     0.9695    0.9244    0.9464       344
           6     0.7525    0.6816    0.7153       223
           7     0.9364    0.9196    0.9279       224
           8     0.8986    0.8160    0.8553       163
           9     0.7081    0.7500    0.7284       152
          10     0.8500    0.5574    0.6733       122

    accuracy                         0.8659      3280
   macro avg     0.8607    0.8340    0.8421      3280
weighted avg     0.8736    0.8659    0.8657      3280


 25%|██▌       | 207/821 [01:50<05:28,  1.87it/s]


KeyboardInterrupt: 

In [ ]:
# 검증 세트를 이용한 모델 평가
model.eval()
total_eval_accuracy = 0
for batch in tqdm(val_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    labels = batch["labels"]

    # 정확도 계산
    accuracy = (predictions == labels).cpu().numpy().mean() * 100
    total_eval_accuracy += accuracy

# 에폭당 평균 검증 정확도 계산
avg_val_accuracy = total_eval_accuracy / len(val_loader)
print(f"Validation Accuracy: {avg_val_accuracy:.2f}%")